In [53]:
import xarray as xr
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

#AO Data 

In [2]:
AOData = xr.open_dataset('../../../../Data/AO_EU/AO_Calculated.nc')
aoTS=AOData.AO
#ao=aoTS[((aoTS.time.dt.month>10) | (aoTS.time.dt.month<2)) & 
#        (aoTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

#EU Data

In [3]:
EUData = xr.open_dataset('../../../../Data/AO_EU/EU.nc')
EUData.EUVal
euTS=EUData.EUVal
#eu=euTS[((euTS.time.dt.month>10) | (euTS.time.dt.month<2)) & 
#        (euTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

#SST Data

In [39]:
sstData=xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/AnomERA5_sst_daily.nc')
sst1=sstData.sst.sel(latitude=slice(5,-5),longitude=slice(-170%360,-120%360)) # nino3.4 region 5S-5N 170W-120W
sstTS=sst1.mean(dim=['latitude','longitude'], skipna=True)  

#Fog Data

In [7]:
fogData = xr.open_dataset('../../../../Data/FogData/CombinedFogData_25Stations.nc')
#plt.figure(figsize=[16,8])
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
y=fg.fogdata.sum(dim='stations').sel(time=slice('1980-1-1','2018-12-31'))
y=y/25.0

#Instantaneos i.e. day before Correlations

In [8]:
AOVar=aoTS.shift(time=1)
ao1D=AOVar[((AOVar.time.dt.month>11) | (AOVar.time.dt.month<2)) & 
           (AOVar.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [9]:
EUVar=euTS.shift(time=1)
eu1D=EUVar[((EUVar.time.dt.month>11) | (EUVar.time.dt.month<2)) & 
           (EUVar.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [41]:
SSTVar=sstTS.shift(time=1)
SST1D=SSTVar[((SSTVar.time.dt.month>11) | (SSTVar.time.dt.month<2)) & 
           (SSTVar.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [10]:
corr_AO = np.corrcoef(y,ao1D)[0,1]
corr_AO

0.004271596099816575

In [11]:
corr_EU = np.corrcoef(y,eu1D)[0,1]
corr_EU

-0.01958108766806569

In [42]:
corr_SST = np.corrcoef(y,SST1D)[0,1]
corr_SST

0.03982708201140668

#5Day Mean Correlations

In [12]:
FivedayMeanAO=aoTS.rolling(time=5).mean()
ao5D=FivedayMeanAO[((FivedayMeanAO.time.dt.month>11) | (FivedayMeanAO.time.dt.month<2)) & 
           (FivedayMeanAO.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [13]:
FivedayMeanEU=euTS.rolling(time=5).mean()
eu5D=FivedayMeanEU[((FivedayMeanEU.time.dt.month>11) | (FivedayMeanEU.time.dt.month<2)) & 
           (FivedayMeanEU.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [43]:
FivedayMeanSST=sstTS.rolling(time=5).mean()
SST5D=FivedayMeanSST[((FivedayMeanSST.time.dt.month>11) | (FivedayMeanSST.time.dt.month<2)) & 
           (FivedayMeanSST.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [14]:
corr_AO5D = np.corrcoef(y,ao5D)[0,1]
corr_AO5D

-0.011599709602252867

In [15]:
corr_EU5D = np.corrcoef(y,eu5D)[0,1]
corr_EU5D

-0.011485224923242176

In [44]:
corr_SST5D = np.corrcoef(y,SST5D)[0,1]
corr_SST5D

0.04055574575695332

#7 Day Mean Correlations

In [16]:
SevendayMeanAO=aoTS.rolling(time=7).mean()
ao7D=SevendayMeanAO[((SevendayMeanAO.time.dt.month>11) | (SevendayMeanAO.time.dt.month<2)) & 
           (SevendayMeanAO.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
SevendayMeanEU=euTS.rolling(time=7).mean()
eu7D=SevendayMeanEU[((SevendayMeanEU.time.dt.month>11) | (SevendayMeanEU.time.dt.month<2)) & 
           (SevendayMeanEU.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [17]:
corr_AO7D = np.corrcoef(y,ao7D)[0,1]
print(corr_AO7D)

corr_EU7D = np.corrcoef(y,eu7D)[0,1]
print(corr_EU7D)

-0.018467348338237582
-0.005535713717814655


In [47]:
SevendayMeanSST=sstTS.rolling(time=7).mean()
SST7D=SevendayMeanSST[((SevendayMeanSST.time.dt.month>11) | (SevendayMeanSST.time.dt.month<2)) & 
           (SevendayMeanSST.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
corr_SST7D = np.corrcoef(y,SST7D)[0,1]
print(corr_SST7D)

0.0405565497931307


#14 Day Mean Correlations

In [18]:
FourteendayMeanAO=aoTS.rolling(time=14).mean()
ao14D=FourteendayMeanAO[((FourteendayMeanAO.time.dt.month>11) | (FourteendayMeanAO.time.dt.month<2)) & 
           (FourteendayMeanAO.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
FourteendayMeanEU=euTS.rolling(time=14).mean()
eu14D=FourteendayMeanEU[((FourteendayMeanEU.time.dt.month>11) | (FourteendayMeanEU.time.dt.month<2)) & 
           (FourteendayMeanEU.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [19]:
corr_AO14D = np.corrcoef(y,ao14D)[0,1]
print(corr_AO14D)

corr_EU14D = np.corrcoef(y,eu14D)[0,1]
print(corr_EU14D)

-0.02734786338749436
0.021315249694369407


In [48]:
FourteendayMeanSST=sstTS.rolling(time=14).mean()
SST14D=FourteendayMeanSST[((FourteendayMeanSST.time.dt.month>11) | (FourteendayMeanSST.time.dt.month<2)) & 
           (FourteendayMeanSST.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
corr_SST14D = np.corrcoef(y,SST14D)[0,1]
print(corr_SST14D)

0.040882610632990944


#30 Day Mean Correlations

In [20]:
ThirtydayMeanAO=aoTS.rolling(time=30).mean()
ao30D=ThirtydayMeanAO[((ThirtydayMeanAO.time.dt.month>11) | (ThirtydayMeanAO.time.dt.month<2)) & 
           (ThirtydayMeanAO.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
ThirtydayMeanEU=euTS.rolling(time=30).mean()
eu30D=ThirtydayMeanEU[((ThirtydayMeanEU.time.dt.month>11) | (ThirtydayMeanEU.time.dt.month<2)) & 
           (ThirtydayMeanEU.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [21]:
corr_AO30D = np.corrcoef(y,ao30D)[0,1]
print(corr_AO30D)

corr_EU30D = np.corrcoef(y,eu30D)[0,1]
print(corr_EU30D)

-0.056287160093617644
0.04078934364679231


In [49]:
ThirtydayMeanSST=sstTS.rolling(time=30).mean()
SST30D=ThirtydayMeanSST[((ThirtydayMeanSST.time.dt.month>11) | (ThirtydayMeanSST.time.dt.month<2)) & 
           (ThirtydayMeanSST.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
corr_SST30D = np.corrcoef(y,SST30D)[0,1]
print(corr_SST30D)


0.047266557799266536


#Input dump 

In [50]:
#Surface data
t2mData = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_t2m_daily_NovDecJan.nc')
d2mData = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_d2m_daily_NovDecJan.nc')
u10Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_u10_daily_NovDecJan.nc')
v10Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_v10_daily_NovDecJan.nc')
# 850 Data
tuv850Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_uvt_daily_NovDecJan.nc')
z850Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_850_z_daily_NovDecJan.nc')
#700 Data
w700Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_700_w_daily_NovDecJan.nc')
# Read Orography
OroData = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')
z500Data = xr.open_dataset('../../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5NH_500_z_daily_NovDecJan.nc')

In [51]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
d2mD = d2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
tuv850D = tuv850Data.sel(latitude=slice(32,24),longitude=slice(74,86))
z850D = z850Data.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = w700Data.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
zD=z500Data.sel(latitude=slice(32,24),longitude=slice(74,86))

In [54]:
# Calculate wind speed and relative humidity
ws = ((v10D.v10.values**2)+(u10D.u10.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
rh = (e**((17.625*(d2mD.d2m.values-273.15))/(243.04+(d2mD.d2m.values-273.15)))/e**((17.625*(t2mD.t2m.values-273.15))/(243.04+(t2mD.t2m.values-273.15))))
rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rh)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
#Calculate inv
inv=t2mD.t2m.values-tuv850D.t.values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'
#u shear calculation
ushear=(tuv850D.u.values-u10D.u10.values)/(z850D.z.values/9.81) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [55]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [56]:
# Apply Mask 
t2mD.t2m.values=t2mD.t2m.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.w.values=w700D.w.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask
zD.z.values=zD.z.values*mask

In [57]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).t2m
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).w
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear
zTS1=zD.mean(dim=['latitude','longitude'], skipna=True).z

In [58]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)
zTS=zTS1.shift(time=1)


In [59]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
z=zTS[((zTS.time.dt.month>11) | (zTS.time.dt.month<2)) & (zTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))

In [72]:
# input X
X       = pd.DataFrame([t2m.values,ws.values,rh.values,inv.values,w.values,ushear.values*100.0,ao1D.values,
                        eu1D.values,SST1D.values,ao5D.values,eu5D.values,SST5D.values,
                       ao7D.values,eu7D.values,SST7D.values,ao14D.values,eu14D.values,SST14D.values, 
                       ao30D.values,eu30D.values,SST30D.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear','ao1', 'eu1', 'sst1',
                             'ao5', 'eu5', 'sst5','ao7', 'eu7', 'sst7',
                             'ao14', 'eu14', 'sst14','ao30', 'eu30', 'sst30']).T[:]
#X       = pd.DataFrame([ws.values,rh.values,inv.values,w.values]).T[:]
X.index = pd.to_datetime(t2m.time.values)
X

,t2m,ws,rh,inv,w,ushear,ao1,eu1,sst1,ao5,...,sst5,ao7,eu7,sst7,ao14,eu14,sst14,ao30,eu30,sst30
1980-01-01 09:00:00,289.120636,1.737170,0.681204,5.449228,-0.103194,0.353420,-1.750939,0.326036,0.117789,-1.214464,...,0.223350,-0.665203,-0.062344,0.287552,0.153002,0.001658,0.409895,0.412610,-0.017662,0.348855
1980-01-02 09:00:00,288.971771,2.071385,0.743338,5.749483,-0.125205,0.273085,-1.824208,0.821619,0.081307,-1.500719,...,0.155103,-0.987637,-0.018502,0.230905,0.036301,0.034050,0.374667,0.278477,0.033050,0.362719
1980-01-03 09:00:00,288.303253,1.375861,0.758891,6.781636,-0.049128,0.160400,-1.439631,0.484874,0.058236,-1.521642,...,0.111137,-1.215665,0.040600,0.181082,-0.121597,0.036583,0.343331,0.159585,0.070172,0.372381
1980-01-04 09:00:00,287.083527,1.883535,0.746639,6.586329,0.070247,0.053234,-0.997704,0.013497,0.092586,-1.421591,...,0.101261,-1.370968,0.184171,0.146355,-0.324608,0.068139,0.315898,0.040884,0.120136,0.379705
1980-01-05 09:00:00,286.249054,1.991354,0.709822,5.626440,0.038802,0.038249,-1.095475,0.433607,0.156388,-1.297015,...,0.123725,-1.404535,0.351457,0.134597,-0.505666,0.125147,0.291855,-0.051431,0.157540,0.388022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 09:00:00,287.116974,1.433200,0.588758,4.247632,-0.025993,0.314777,-0.047821,0.367309,0.732377,-0.100498,...,0.748099,-0.163057,0.183831,0.749039,0.224099,0.063579,0.741027,0.257892,0.229153,0.803485
2018-12-28 09:00:00,287.040436,1.897564,0.547522,5.767006,0.163933,0.249560,0.075607,0.866002,0.763326,-0.048009,...,0.745836,-0.125613,0.368613,0.742686,0.140224,0.150347,0.739200,0.277892,0.254875,0.801265
2018-12-29 09:00:00,286.180817,2.137837,0.552296,4.413174,0.137729,0.143824,0.083157,1.291670,0.721219,0.013835,...,0.733624,-0.028950,0.530500,0.735476,0.079405,0.212243,0.735350,0.292222,0.257262,0.796579
2018-12-30 09:00:00,286.110046,1.636086,0.564291,1.974165,0.078003,0.081137,0.216663,0.938830,0.686603,0.109464,...,0.716837,0.028052,0.649130,0.728065,0.055703,0.251367,0.731664,0.302612,0.253413,0.792210


In [73]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X.columns=['t2m', 'ws', 'rh','inv', 'w', 'ushear','ao1', 'eu1',
           'sst1','ao5', 'eu5', 'sst5','ao7', 'eu7', 'sst7',
           'ao14', 'eu14', 'sst14','ao30', 'eu30', 'sst30']
X

,t2m,ws,rh,inv,w,ushear,ao1,eu1,sst1,ao5,...,sst5,ao7,eu7,sst7,ao14,eu14,sst14,ao30,eu30,sst30
1980-01-01 09:00:00,-151.803446,-4.064469,-0.709943,-1.704749,-33.468375,14.908108,-1.503011,0.719577,0.097509,-1.219917,...,0.163287,-0.717822,-0.208186,0.203064,0.219783,0.021537,0.278839,0.792536,-0.122467,0.241408
1980-01-02 09:00:00,-151.844542,-1.727495,9.293493,-1.611327,-38.473908,7.829461,-1.566859,1.811889,0.074668,-1.512209,...,0.120583,-1.074078,-0.056785,0.167653,0.060517,0.182886,0.256840,0.528675,0.295950,0.250061
1980-01-03 09:00:00,-152.029096,-6.590885,11.797603,-1.290181,-21.173405,-2.099535,-1.231729,1.069670,0.060225,-1.533574,...,0.093073,-1.326026,0.147320,0.136508,-0.154973,0.195502,0.237273,0.294796,0.602237,0.256091
1980-01-04 09:00:00,-152.365818,-3.041019,9.825014,-1.350949,5.973481,-11.542335,-0.846622,0.030712,0.081730,-1.431413,...,0.086894,-1.497619,0.643124,0.114801,-0.432028,0.352687,0.220142,0.061290,1.014479,0.260663
1980-01-05 09:00:00,-152.596186,-2.287107,3.897496,-1.649611,-1.177278,-12.862747,-0.931823,0.956673,0.121676,-1.304209,...,0.100949,-1.534708,1.220826,0.107450,-0.679125,0.636649,0.205128,-0.120309,1.323089,0.265853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-27 09:00:00,-152.356585,-6.189951,-15.593767,-2.078616,-15.912171,11.503087,-0.018868,0.810546,0.482291,-0.082453,...,0.491630,-0.163001,0.641949,0.491544,0.316810,0.329969,0.485617,0.488180,1.913961,0.525151
2018-12-28 09:00:00,-152.377714,-2.942925,-22.232640,-1.605875,27.278408,5.756574,0.088690,1.909714,0.501667,-0.028856,...,0.490214,-0.121629,1.280072,0.487573,0.202344,0.762174,0.484476,0.527525,2.126189,0.523765
2018-12-29 09:00:00,-152.615024,-1.262835,-21.464061,-2.027109,21.319440,-3.560168,0.095270,2.847924,0.475305,0.034292,...,0.482573,-0.014826,1.839127,0.483065,0.119341,1.070484,0.482072,0.555714,2.145882,0.520841
2018-12-30 09:00:00,-152.634561,-4.771283,-19.532871,-2.785987,7.737331,-9.083744,0.211610,2.070233,0.453632,0.131938,...,0.472069,0.048156,2.248801,0.478433,0.086995,1.265365,0.479770,0.576153,2.114124,0.518113


In [74]:
dump(X, 'X_All.joblib') 
dump(y,'y_All.joblib')

['y_All.joblib']